Juan Pablo Márquez 2022

# Càrrega de dades mongodb

* Documenta el procés amb un script mongosh.
* A partir de les següents data sets de dades: https://www.kaggle.com/stefanoleone992/fifa-22-complete-player-dataset
* Carrega les dades a un base de dades i en un col·lecció de jugadors.

Para comenzar a trabajar sobre la base de datos de jugadores del fifa 22:
1. Crear una base de datos a través de MongoDbCompass.
2. Importar los datos de players22.csv a la DB.
3. Una vez creada la DB con su colección y importados los datos de los jugadores dentro de ella, simplente accedemos a la consola de MongoDbCompass y ejecutamos el comando 'use fifa22'.

In [2]:
# Para comenzar a trabajar sobre la base de datos de jugadores del fifa 22:
# 1. Crear una base de datos a través de MongoDbCompass. Será con la que trabajamos en este notebook.
# 2. Importar los datos de players22.csv a la DB. En la columna wage_eur transformarla a 'Number' / int.
# 3. Importar los paquetes necesarios para realizar las conexionces y consultas, entre otras cosas.

import pymongo 
from pymongo import MongoClient
import json

# Una vez completados los pasos previos nos conectamos a la DB, yo la he nombrado fifa22

client = MongoClient('localhost',27017)
db = client["fifa22"]
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'fifa22')

In [3]:
# Comprobamos que la conexion a la DB es correcta y que tenga la coleccion 'players'. 
db.list_collection_names()

['players']

# Consultar la col·lecció de jugadors. Consultes simples.

1. Quins jugadors juguen al Mallorca? Mostra només el nom.

    db.players.find({"club_name":"RCD Mallorca"},{"long_name":1})
    
2. Quins jugadors juguen a la primera divisió i a la segona a Espanya? Mostra només el nom.
    
    db.players.find({$or: [{"league_name":"Spain Primera Division"}, {"league_name":"Spanish Segunda División"}]},{"long_name":1})
  
3. Quins son els equips de primera divisió i de segona a Espanya? Mostra a la divisió que pertanyen.

    db.players.aggregate([{$match: {"league_name": {$in: ["Spain Primera Division","Spanish Segunda División"]}}},{$group:{"_id": {"club_name": "$club_name","league_name": "$league_name"}}},{$sort: {"club_name": -1,"_id": 1}}])

# Consultar la col·lecció de jugadors. Consultes d’agregació.

1. Quin és el valor màxim del camp “power_shot_power”?

    db.players.aggregate([{ $group : { _id: "power_shot_power", max: { $max : "$power_shot_power" }}}])
    
2. Quin és el jugador amb el valor màxim el camp “power_shot_power”? Mostra el nom i l’equip al que pertany.
    
    db.players.aggregate([{$group:{_id: "$long_name","equipo": {$first: "$club_name"},"potenciaTiro": { $max : "$power_shot_power" },}},{$sort: {"potenciaTiro" : -1, _id: 1}},{$limit: 1}])
  
3. Quin és el jugador amb el valor màxim el camp “power_shot_power” del RCD Mallorca i de la UD Ibiza? Mostra el nom i l’equip al que pertany.

    db.players.aggregate([{$match: {"club_name": {$in : ["RCD Mallorca","UD Ibiza"]}}},{$group: {"equipo": { $first : "$club_name"},"potencia de tiro": { $max : "$power_shot_power" },_id: "$long_name"}},{$sort: {"potencia de tiro" : -1, _id: 1}},{$limit: 2}])

4. Equips que la suma del sous del seus jugadors és superior a 3000000.
    
    db.players.aggregate([{$group: {'_id': '$club_name','Sueldos': {$sum: '$wage_eur'}}}, {$match: {'Sueldos': {$gt: 3000000}}}, {$sort: {'Sueldos': -1}}])
    
5. Dels equips que la suma del sous del seus jugadors és superior a 3000000. Mostra els jugadors amb el sou més alt.
    
    db.players.aggregate([{'$group': {'_id': '$club_name','Total Sueldo Equipo': {'$sum': '$wage_eur'},'Jugador': {'$first': '$long_name'},'Sueldo': {'$max': '$wage_eur'}}}, {'$match': {'Total Sueldo Equipo': {'$gt': 3000000}}}, {'$sort': {'_id': -1}}])
    
6. Dels equips que la suma del sous del seus jugadors és superior a 3000000. Mostra els jugadors que el seu sou és superior a la mitjana del seu equip.
    
    db.players.aggregate({$group:{_id: '$club_name',sou: { $sum: '$wage_eur'},mitja: {$avg: '$wage_eur'},jugadors: {$push: {short_name:'$short_name', wage_eur:'$wage_eur'}}}},{$match:{sou: {$gt: 3000000}}},{$project:{"jugadors":{"$filter":{"input":"$jugadors","as":"jug","cond":{"$gt": ["$$jug.wage_eur", "$mitja"]}}},club: "$club_name",mitja: "$mitja"}})

# Modificar la col·lecció de jugadors.

1. Afegeix un camp que sigui la suma dels camps "power_shot_power", "power_jumping", "power_stamina", "power_strength" i “power_long_shots".

    db.players.find().forEach( function(player){SumaPowers = parseFloat(player.power_jumping) + parseFloat(player.power_shot_power) + parseFloat(player.power_stamina) + parseFloat(player.power_strength) + parseFloat(player.power_long_shots);db.players.updateOne({_id: player._id},{$set:{'SumaPowers':SumaPowers}})})

2. De la col·lecció jugador agafa els atributs: short_name, player_positions, value_eur, club_name, player_tags, i els 10 camps que creguis que estan relacionats amb el valor del jugador. Esborra la resta.
    
    var arr = db.players.find({},{short_name:1,player_positions:1,value_eur:1,club_name:1,player_tags:1,long_name:1,overall:1,league_name:1,wage_eur:1,age:1,pace:1,shooting:1,passing:1,dribbling:1,defending:1})
    
    db.players.deleteMany({})
    
    arr.forEach(function(player){db.players.insertOne(player)})
    
3. Esborra els judadors que el seu valor sigui inferior a 30000000.
    
    db.players.remove({value_eur : {$lt: '30000000'}})


# Utilitzar python per accedir a mongodb.

1. Convertir el camp player_tag en un llista.

In [ ]:
# Convertir el camp player_tag en un llista.
import pymongo
import json

client = MongoClient('localhost',27017)
db = client["fifa22"]

lista = list(db.players.find())

for player in lista:
    if 'player_tags' in player:
        tagList = player['player_tags']
        db.players.update_one({'_id':player['_id']},{'$set':{'player_tags':tagList}})

list(db.players.find().limit(1))